<h1>Contenido<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#0.--Librerías" data-toc-modified-id="0.--Librerías-0">0.  Librerías</a></span></li><li><span><a href="#1.-Preprocesamiento" data-toc-modified-id="1.-Preprocesamiento-1">1. Preprocesamiento</a></span><ul class="toc-item"><li><span><a href="#1.1-Perfilamiento--de-los-archivos-a-usar" data-toc-modified-id="1.1-Perfilamiento--de-los-archivos-a-usar-1.1">1.1 Perfilamiento  de los archivos a usar</a></span></li><li><span><a href="#1.2-Selección-de-variables-y-transformaciones-de-columnas" data-toc-modified-id="1.2-Selección-de-variables-y-transformaciones-de-columnas-1.2">1.2 Selección de variables y transformaciones de columnas</a></span></li></ul></li><li><span><a href="#2.-Procesamiento" data-toc-modified-id="2.-Procesamiento-2">2. Procesamiento</a></span><ul class="toc-item"><li><span><a href="#2.1-Cuadro-1" data-toc-modified-id="2.1-Cuadro-1-2.1">2.1 Cuadro 1</a></span></li><li><span><a href="#2.2-Cuadro-2" data-toc-modified-id="2.2-Cuadro-2-2.2">2.2 Cuadro 2</a></span></li><li><span><a href="#2.3-Tabla-3" data-toc-modified-id="2.3-Tabla-3-2.3">2.3 Tabla 3</a></span></li></ul></li></ul></div>

## 0.  Librerías

In [1]:
%matplotlib inline
from pandas_profiling import ProfileReport
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## 1. Preprocesamiento

Se define un array de meses y una url base para leer archivos

In [2]:
meses = ['Enero','Febrero','Marzo','Abril','Mayo','Junio','Julio','Agosto','Septiembre','Octubre','Noviembre','Diciembre']
filesUrlBase = 'https://raw.githubusercontent.com/jcmontoyar/CUSO2020/master/Data/'

Método genérico para leer cualquier archivo del censo, recibe el mes, el area y la categoria.
Retorna un dataframe del archivo escogido

In [3]:
def readMonthAreaCategory(mes, area, category):
    fileUrl = filesUrlBase + mes + '.csv' + '/' + area + ' - ' + category + '.csv'
    fileUrl = fileUrl.replace(" ", "%20")
    return pd.read_csv(fileUrl,delimiter=';', encoding='utf-8')

### 1.1 Perfilamiento  de los archivos a usar
Perfilamiento cabecera caracteristicas generales

In [12]:
ProfileReport(readMonthAreaCategory('Enero', 'Cabecera' ,'Caracteristicas generales (Personas)'))

C:\Users\julia\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3249: DtypeWarning: Columns (37) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
C:\Users\julia\Anaconda3\lib\site-packages\pandas_profiling\describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


Number of variables,38
Number of observations,56644
Total Missing (%),0.0%
Total size in memory,16.4 MiB
Average record size in memory,304.0 B
Numeric,11
Categorical,22
Boolean,1
Date,0
Text (Unique),0
Rejected,4


Perfilamiento ocupados

In [13]:
ProfileReport(readMonthAreaCategory('Enero', 'Cabecera' ,'Ocupados'))

C:\Users\julia\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3249: DtypeWarning: Columns (96,165) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
C:\Users\julia\Anaconda3\lib\site-packages\pandas_profiling\describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


Number of variables,166
Number of observations,24869
Total Missing (%),0.0%
Total size in memory,31.5 MiB
Average record size in memory,1.3 KiB
Numeric,18
Categorical,128
Boolean,11
Date,0
Text (Unique),0
Rejected,9


Perfilamiento vivienda y hogares

In [4]:
ProfileReport(readMonthAreaCategory('Enero', 'Cabecera' ,'Vivienda y Hogares'))

C:\Users\julia\Anaconda3\lib\site-packages\pandas_profiling\describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


Number of variables,60
Number of observations,17227
Total Missing (%),0.0%
Total size in memory,7.9 MiB
Average record size in memory,480.0 B
Numeric,14
Categorical,14
Boolean,27
Date,0
Text (Unique),0
Rejected,5


### 1.2 Selección de variables y transformaciones de columnas
Métodos para leer archivos de cabecera a usar y transformacion de columnas para el procesamiento

In [5]:
# Array con los estratos 
estratos = ['Sin conexion, pirata y estrato 1', 'Estrato 2', 'Estrato 3', 'Estrato 4, 5 y 6','Sin informacion']

# Recibe un dataframe de categoria ocupados y le añade una columna indicando el estrato
def setEstrato(df):
    conditions = [
        (df['P4030S1'] == 1) & (df['P4030S1A1'] == 0),
        (df['P4030S1'] == 1) & (df['P4030S1A1'] == 1),
        (df['P4030S1'] == 2),
        (df['P4030S1'] == 1) & (df['P4030S1A1'] == 2),
        (df['P4030S1'] == 1) & (df['P4030S1A1'] == 3),
        (df['P4030S1'] == 1) & (df['P4030S1A1'] == 4),
        (df['P4030S1'] == 1) & (df['P4030S1A1'] == 5),
        (df['P4030S1'] == 1) & (df['P4030S1A1'] == 6),
        (df['P4030S1'] == 1) & (df['P4030S1A1'] == 9)]
    choices = [estratos[0], estratos[0], estratos[0], estratos[1],estratos[2],estratos[3],estratos[3],estratos[3], estratos[4]]
    df['Estrato'] = np.select(conditions, choices, default = estratos[4])
    return df

# Lee y procesa archivos de cabecera - ocupados dado un mes
def readOcupadosCabecera(mes):
    #Obtenener data frame
    df = readMonthAreaCategory(mes, 'Cabecera', 'Ocupados')
    
    #Delete unnecessary columns
    df = df.drop(['MES','CLASE','REGIS','P7050S1','P550','P7050S1','OCI', 'HOGAR','RAMA4D','DPTO'],axis=1)
    
    # Transfrom expantion factor to float
    df['fex_c_2011'] = df['fex_c_2011'].str.replace(',', '.').astype(float)
    
    return df

# Read vivienda y hogares given the month.
def readViviendaYHogaresCabecera(mes):
    #Obtenener data frame
    df = readMonthAreaCategory(mes, 'Cabecera', 'Vivienda y Hogares')
    
    #Fix null value in P4030S1A1 with 9 (no information)
    df['P4030S1A1'] = df['P4030S1A1'].replace(to_replace =" ",  value ="9").astype(int)
    
    #Delete unnecessary columns
    df = df.drop(['MES','HOGAR','clase','REGIS'],axis=1)
    
    # Transfrom expantion factor to float
    df['fex_c_2011'] = df['fex_c_2011'].str.replace(',', '.').astype(float)
    
    #Set estratos
    df = setEstrato(df)
    return df

# Lee archivos de cabecera de caracterisiticas generales dado un mes
def readCaracteristicasGeneralesCabecera(mes):
    #Obtenener data frame
    df = readMonthAreaCategory(mes, 'Cabecera', 'Caracteristicas generales (Personas)')
    
    #Delete unnecessary columns
    df = df.drop(['MES','HOGAR','CLASE','REGIS'],axis=1)
    
    # Transfrom expantion factor to float
    df['fex_c_2011'] = df['fex_c_2011'].str.replace(',', '.').astype(float) 
    
    return df

Crear array de dataframes de todos los archivos de vivienda y hogares de cabecera

In [6]:
dfs = []
for i in meses:
    dfs.append(readViviendaYHogaresCabecera(i))

C:\Users\julia\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3326: DtypeWarning: Columns (47) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\julia\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3326: DtypeWarning: Columns (5,47) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\julia\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3326: DtypeWarning: Columns (47,53) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Crear array de dataframes de todos los archivos de ocupados de cabecera

In [7]:
dfOcupados = []
for i in meses:
    dfOcupados.append(readOcupadosCabecera(i))

C:\Users\julia\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3326: DtypeWarning: Columns (96,165) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\julia\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3326: DtypeWarning: Columns (165) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\julia\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3326: DtypeWarning: Columns (96) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Crear array de dataframes de todos los archivos de caractetisticas generales de cabecera

In [8]:
dfCaracteristicasGen = []
for i in meses:
    dfCaracteristicasGen.append(readCaracteristicasGeneralesCabecera(i))

C:\Users\julia\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3326: DtypeWarning: Columns (37) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


## 2. Procesamiento

### 2.1 Cuadro 1

Función que calcula los hogares por estrado dado el numero de meses a analizar

In [14]:
def calculateNumberOfHogaresByEstrato(df, numMeses):
    #Contador
    aux = 0
    
    #Contador por estrato, cada indice representa la cuenta del estrato con el mismo orden que el array de estratos
    #En el último índice se encuentra la suma de los demás
    hogaresPorEstrato = [0,0,0,0,0,0]
    
    #Aux de directorio para contar solo por hogar
    directorioAnterior = -1
    
    #Hogar actual
    hogarAnterior = "init"
    for i in df['DIRECTORIO']:
        
        #Obtenemos directorio actual
        dirActual = i
        
        #Obtenemos el hogar (directorio + secuencia)
        hogarActual = str(dirActual) + str(df['SECUENCIA_P'][aux])
        
        #Si el directorio anterior es distinto al actual significa que estamos en otro hogar
        if hogarAnterior != hogarActual:
            
            #Obtener estrato actual
            estratoActual = df['Estrato'][aux]
            
            #Obtener factor de expansion segun el numero de meses para el cual se está haciendo el consolidado
            factorExp = float(df['fex_c_2011_x'][aux])/numMeses
            
            #Sumar al total
            hogaresPorEstrato[5] += factorExp
            
            #Se suma al array según el caso
            if estratoActual == estratos[0]:
                hogaresPorEstrato[0] += factorExp
            elif estratoActual == estratos[1]:
                hogaresPorEstrato[1] += factorExp
            elif estratoActual == estratos[2]:
                hogaresPorEstrato[2] += factorExp
            elif estratoActual == estratos[3]:
                hogaresPorEstrato[3] += factorExp
            elif estratoActual == estratos[4]:
                hogaresPorEstrato[4] += factorExp
        
        #Actualizar aux y directorio anterior
        aux += 1
        hogarAnterior = hogarActual
    return hogaresPorEstrato

Obtener meses para los cuales calculas los hogares

In [15]:
mesesTabla1 = input('Ingresar meses separados por - (numero del mes) \n')

Ingresar meses separados por - (numero del mes) 
7-8-9


In [16]:
mesesTabla1 = mesesTabla1.split('-')

In [21]:
# Array con los estratos 
estratos = ['Sin conexion, pirata y estrato 1', 'Estrato 2', 'Estrato 3', 'Estrato 4, 5 y 6','Sin informacion']

# Columna de estratos a poner en la tabla
estratosTabla = estratos
estratosTabla.append('Total HOGARES')

#Array con contador por estrato, siguiendo el orden de la columna estratos por tabla
numHogares = [0,0,0,0,0,0]

#Calcular suma de hogares por estrato por cada mes en array meses tabla 1
for i in mesesTabla1:
    #Obtener dataframes del mes i necesarios para calcular los hogares por estrato
    dfVivienda =dfs[int(i)-1]
    dfGen =dfCaracteristicasGen[int(i)-1]
    
    #Hacer join de los dataframes
    joined = pd.merge(dfGen, dfVivienda, on=['DIRECTORIO','SECUENCIA_P'], how='outer')
    
    #Obtener numero de los hogares y sumarlo al array de num hogares
    numHogares = np.add(numHogares, calculateNumberOfHogaresByEstrato(joined, len(mesesTabla1)))

    
#Calcular porcentaje de errores
porcentajeHogares = [0,0,0,0,0,0]
for i in range(len(porcentajeHogares)):
    porcentajeHogares[i] = numHogares[i]/numHogares[5]*100
    
#Construir table data del dataframe para imprimirlo
dataTabla = {'Estrato':  estratosTabla,
        'No. de hogares': numHogares,
        'Porcentaje de Hogares': porcentajeHogares,
        }

#Construir dataframe y transformar los resultados a string para que sea sencillo copiarlos al excel
tabla = pd.DataFrame(dataTabla, columns = ['Estrato' ,'No. de hogares','Porcentaje de Hogares'])
tabla['Porcentaje de Hogares'] = tabla['Porcentaje de Hogares'].astype(str)
tabla['No. de hogares'] = tabla['No. de hogares'].astype(str)
tabla

,Estrato,No. de hogares,Porcentaje de Hogares
0,"Sin conexion, pirata y estrato 1",3170094.7160020513,26.961836035330638
1,Estrato 2,4721846.807765538,40.15957591812264
2,Estrato 3,2737318.948921386,23.28105349807922
3,"Estrato 4, 5 y 6",1083118.9495509572,9.211988328658368
4,Sin informacion,45331.40964843126,0.3855462198091049
5,Total HOGARES,11757710.831888366,100.0


Correr para copiar la tabla al clipboard

In [22]:
tabla.to_clipboard(excel=True)

### 2.2 Cuadro 2

Metodo que calcula el numero de personas por estrato dado un dataframe y el numero de meses para la consolidación

In [29]:
def calculatePeopleByEstrato(df, numMeses):
    #Contador
    aux = 0
    
    #Contador por estrato, cada indice representa la cuenta del estrato con el mismo orden que el array de estratos
    #En el último índice se encuentra la suma de los demás
    personasPorEstrato = [0,0,0,0,0,0]
    
    for i in df['DIRECTORIO']:
        
        #Obtenemos directorio actual
        dirActual = i
        
        #Obtener estrato actual
        estratoActual = df['Estrato'][aux]
            
        #Obtener factor de expansion segun el numero de meses para el cual se está haciendo el consolidado
        factorExp = df['fex_c_2011_x'][aux]/numMeses
            
        #Sumar al total
        personasPorEstrato[5] += factorExp
            
        #Se suma al array según el caso
        if estratoActual == estratos[0]:
            personasPorEstrato[0] += factorExp
        elif estratoActual == estratos[1]:
            personasPorEstrato[1] += factorExp
        elif estratoActual == estratos[2]:
            personasPorEstrato[2] += factorExp
        elif estratoActual == estratos[3]:
            personasPorEstrato[3] += factorExp
        elif estratoActual == estratos[4]:
            personasPorEstrato[4] += factorExp
        
        #Actualizar aux y directorio anterior
        aux += 1 
    return personasPorEstrato

In [25]:
mesesTabla2 = input('Ingresar meses separados por - (numero del mes) \n')

Ingresar meses separados por - (numero del mes) 
7-8-9


In [26]:
mesesTabla2 = mesesTabla2.split('-')

In [31]:
# Array con los estratos 
estratos = ['Sin conexion, pirata y estrato 1', 'Estrato 2', 'Estrato 3', 'Estrato 4, 5 y 6','Sin informacion']

# Columna de estratos a poner en la tabla
estratosTabla = estratos
estratosTabla.append('Total PERSONAS')

#Array con contador por estrato, siguiendo el orden de la columna estratos por tabla
numPersonas = [0,0,0,0,0,0]

#Calcular suma de personas por estrato por cada mes en array meses tabla 2
for i in mesesTabla2:
    #Obtener dataframes del mes i necesarios para calcular los hogares por estrato
    dfVivienda =dfs[int(i)-1]
    dfGen =dfCaracteristicasGen[int(i)-1]
    
    #Hacer join de los dataframes
    joined = pd.merge(dfGen, dfVivienda, on=['DIRECTORIO','SECUENCIA_P'], how='outer')
    
    #Obtener numero de las personas y sumarlo al array de num personas
    numPersonas = np.add(numPersonas, calculatePeopleByEstrato(joined, len(mesesTabla2)))

    
#Calcular porcentaje de personas
porcentajePersonas = [0,0,0,0,0,0]
for i in range(len(porcentajePersonas)):
    porcentajePersonas[i] = numPersonas[i]/numPersonas[5]*100
    
#Construir table data del dataframe para imprimirlo
dataTabla = {'Estrato':  estratosTabla,
        'No. de personas': numPersonas,
        'Porcentaje de Personas': porcentajePersonas,
        }

#Construir dataframe y transformar los resultados a string para que sea sencillo copiarlos al excel
tabla = pd.DataFrame(dataTabla, columns = ['Estrato' ,'No. de personas','Porcentaje de Personas'])
tabla['Porcentaje de Personas'] = tabla['Porcentaje de Personas'].astype(str)
tabla['No. de personas'] = tabla['No. de personas'].astype(str)
tabla

,Estrato,No. de personas,Porcentaje de Personas
0,"Sin conexion, pirata y estrato 1",11536045.93367958,30.39123937114563
1,Estrato 2,15252714.920889836,40.18265122083752
2,Estrato 3,8255777.5008479655,21.74950686445125
3,"Estrato 4, 5 y 6",2777805.6624083947,7.318014967876437
4,Sin informacion,136114.31550755908,0.35858757569201316
5,Total HOGARES,37958458.333332255,100.0


Ejecutar para copiar tabla 2 al clipboard

In [32]:
tabla.to_clipboard(excel=True)

### 2.3 Tabla 3

In [47]:
PET = 12
colEdad = 'P6040'
colSexo = 'P6020'

In [59]:
def calculateGenderByEstratoPET(df, numMeses):
    #Contador
    aux = 0
    
    #Contador por genero por estrato. Primer array de hombres, segundo de mujeres, tercero consolidados
    #En el último índice se encuentra la suma de los demás
    hombresMujeresPorEstrato = [[0,0,0,0,0,0],[0,0,0,0,0,0],[0,0,0,0,0,0]]
    for i in df[colSexo]:
        
        #Obtenemos sexo actual
        sex = i
        
        #Obtener edad actual
        edad = df[colEdad][aux]
        
        #Obtener estrato actual
        estratoActual = df['Estrato'][aux]
            
        #Obtener factor de expansion segun el numero de meses para el cual se está haciendo el consolidado
        factorExp = df['fex_c_2011_x'][aux]/numMeses
        
        #Indica el indice para guardar, queda por defecto en indice 1 (mujer)
        sumarEn = 1
        
        #Si es 1, entonces es hombre
        if sex == 1:
            sumarEn = 0
        if edad >= PET: 
            #Se suma al array según el caso
            if estratoActual == estratos[0]:
                hombresMujeresPorEstrato[sumarEn][0] += factorExp
            elif estratoActual == estratos[1]:
                hombresMujeresPorEstrato[sumarEn][1] += factorExp
            elif estratoActual == estratos[2]:
                hombresMujeresPorEstrato[sumarEn][2] += factorExp
            elif estratoActual == estratos[3]:
                hombresMujeresPorEstrato[sumarEn][3] += factorExp
            elif estratoActual == estratos[4]:
                hombresMujeresPorEstrato[sumarEn][4] += factorExp
        
        #Actualizar aux y directorio anterior
        aux += 1 
    hombresMujeresPorEstrato[0][5] = np.sum(hombresMujeresPorEstrato[0])
    hombresMujeresPorEstrato[1][5] = np.sum(hombresMujeresPorEstrato[1])
    for i in range(len(hombresMujeresPorEstrato[2])):
        hombresMujeresPorEstrato[2][i] = hombresMujeresPorEstrato[1][i] + hombresMujeresPorEstrato[0][i]
    print(hombresMujeresPorEstrato)   
    return hombresMujeresPorEstrato

In [39]:
mesesTabla3 = input('Ingresar meses separados por - (numero del mes) \n')

Ingresar meses separados por - (numero del mes) 
7-8-9


In [41]:
mesesTabla3 = mesesTabla3.split('-')

In [60]:
# Array con los estratos 
estratos = ['Sin conexion, pirata y estrato 1', 'Estrato 2', 'Estrato 3', 'Estrato 4, 5 y 6','Sin informacion']

# Columna de estratos a poner en la tabla
estratosTabla = estratos
estratosTabla.append('Total PET')

#Array con contador por estrato, siguiendo el orden de la columna estratos por tabla
results = [[0,0,0,0,0,0],[0,0,0,0,0,0],[0,0,0,0,0,0]]

#Calcular suma de personas por estrato por cada mes en array meses tabla 2
for i in mesesTabla3:
    #Obtener dataframes del mes i necesarios para calcular los hogares por estrato
    dfVivienda =dfs[int(i)-1]
    dfGen =dfCaracteristicasGen[int(i)-1]
    
    #Hacer join de los dataframes
    joined = pd.merge(dfGen, dfVivienda, on=['DIRECTORIO','SECUENCIA_P'], how='outer')
    
    #Obtener numero de las personas y sumarlo al array de num personas
    results = np.add(results, calculateGenderByEstratoPET(joined, len(mesesTabla3)))

    
#Calcular porcentaje de personas
porcentajePersonas = [[0,0,0,0,0,0],[0,0,0,0,0,0],[0,0,0,0,0,0]]
for i in range(len(porcentajePersonas)):
    for j in range(len(porcentajePersonas[i])):
        porcentajePersonas[i][j] = results[i][j]/results[i][5]*100
    
#Construir table data del dataframe para imprimirlo
dataTabla = {'Estrato':  estratosTabla,
        'Numero hombres': results[0],
        'Porcentaje hombres': porcentajePersonas[0],
        'Numero mujeres': results[1],
        'Porcentaje mujeres': porcentajePersonas[1],
        'Numero total': results[2],
        'Porcentaje total': porcentajePersonas[2],
        }

#Construir dataframe y transformar los resultados a string para que sea sencillo copiarlos al excel
tabla = pd.DataFrame(dataTabla, columns = ['Estrato' ,'Numero hombres','Porcentaje hombres',
                                           'Numero mujeres','Porcentaje mujeres','Numero total','Porcentaje total'])
#tabla['Porcentaje de Personas'] = tabla['Porcentaje de Personas'].astype(str)
#tabla['No. de personas'] = tabla['No. de personas'].astype(str)
tabla

[[1594854.8914629787, 1767682.655868149, 1103126.3178398875, 373350.0391972846, 28867.42896504181, 4867881.333333341], [1690771.2356536435, 1936345.028524997, 1240566.5763126835, 447175.9244344078, 27879.568407608374, 5342738.33333334], [3285626.1271166224, 3704027.684393146, 2343692.894152571, 820525.9636316923, 56746.997372650185, 10210619.666666681]]
[[1390645.3488011821, 2038783.026047531, 1021532.4984847695, 415705.3879400652, 7093.072059785593, 4873759.333333334], [1471173.3916498597, 2208461.443980717, 1179262.3067234498, 484032.06008911174, 6011.464223542879, 5348940.666666681], [2861818.7404510416, 4247244.470028248, 2200794.805208219, 899737.4480291769, 13104.536283328473, 10222700.000000015]]
[[1290443.4312569138, 2081852.7469584772, 1143591.2855395612, 346647.8285222761, 17100.707722767107, 4879635.999999995], [1385499.4468514824, 2243628.9801966534, 1309487.8228125258, 402575.640955889, 13954.775850127799, 5355146.666666677], [2675942.878108396, 4325481.72715513, 2453079.1

,Estrato,Numero hombres,Porcentaje hombres,Numero mujeres,Porcentaje mujeres,Numero total,Porcentaje total
0,"Sin conexion, pirata y estrato 1",4.275944e+06,29.244667,4.547444e+06,28.338590,8.823388e+06,28.770570
1,Estrato 2,5.888318e+06,40.272259,6.388435e+06,39.811210,1.227675e+07,40.031019
2,Estrato 3,3.268250e+06,22.352700,3.729317e+06,23.240215,6.997567e+06,22.817084
3,"Estrato 4, 5 y 6",1.135703e+06,7.767470,1.333784e+06,8.311822,2.469487e+06,8.052298
4,Sin informacion,5.306121e+04,0.362904,4.784581e+04,0.298164,1.009070e+05,0.329029
5,Total PET,1.462128e+07,100.000000,1.604683e+07,100.000000,3.066810e+07,100.000000


In [58]:
tabla.to_clipboard(excel=True)